In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

from mypipes_facebook import *

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/facebook_comments.csv')

In [3]:
df

,likes,checkins,talking_about,page_category,feat5,feat6,feat7,feat8,feat9,feat10,...,CC4,CC5,Base_Time,Post_Length,Post Share Count,Post Promotion Status,H local,Comments_in_next_H_hrs,Post Published Weekday,Base Date Time Weekday
0,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,65,166,2,0,24,0,Wednesday,Saturday
1,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,10,132,1,0,24,0,Thursday,Friday
2,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,14,133,2,0,24,0,Friday,Saturday
3,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,7,-3,62,131,1,0,24,0,Friday,Monday
4,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,58,142,5,0,24,0,Monday,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,7170111,70,497000,Professional sports team,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,240,-44,56,12,1511,0,2,1,Wednesday,Friday
40945,7170111,70,497000,Professional sports team,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,619,-86,53,149,1099,0,3,2,Wednesday,Friday
40946,7170111,70,497000,Professional sports team,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,620,620,19,120,2962,0,24,72,Wednesday,Thursday
40947,7170111,70,497000,Professional sports team,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,588,-251,33,33,1383,0,9,28,Wednesday,Friday


In [4]:
x = df.drop('Comments_in_next_H_hrs', axis = 1)
y = df[['Comments_in_next_H_hrs']]

In [5]:
y

,Comments_in_next_H_hrs
0,0
1,0
2,0
3,0
4,0
...,...
40944,1
40945,2
40946,72
40947,28


In [6]:
df['Post_Length'].value_counts()

0       4584
14       300
53       286
11       271
58       230
        ... 
876        1
1888       1
1281       1
1734       1
637        1
Name: Post_Length, Length: 1477, dtype: int64

In [7]:
'''
P1:
page_category: impute missing values, create dummy

P2 & P3:
Post Published Weekday, Base Date Time Weekday: cyclic features 


P4:
likes, checkins, talking_about, feat5, feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat14,feat15,
feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29, CC1, CC2, CC3, 
CC4, CC5,Base_Time, Post_Length, Post Share Count, Post Promotion Status, H local 

'''

'\nP1:\npage_category: impute missing values, create dummy\n\nP2 & P3:\nPost Published Weekday, Base Date Time Weekday: cyclic features \n\n\nP4:\nlikes, checkins, talking_about, feat5, feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat14,feat15,\nfeat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29, CC1, CC2, CC3, \nCC4, CC5,Base_Time, Post_Length, Post Share Count, Post Promotion Status, H local \n\n'

In [8]:
p1 = pdPipeline([
    ('var_select', VarSelector(['page_category'])),
    ('missing_trt', DataFrameImputer()),
    ('create_dummy', dummy(200))
])

p2 = pdPipeline([
    ('var_select', VarSelector(['Post Published Weekday', 'Base Date Time Weekday'])),
    ('wday_to_num', wday_conversion())
])

p3 = pdPipeline([
    ('var_select', VarSelector(['Post Published Weekday', 'Base Date Time Weekday'])),
    ('wday_to_cyclic', cyclic_features())
])

p4 = pdPipeline([
    ('var_select', VarSelector(['likes','checkins','talking_about','feat5','feat6','feat7','feat8','feat9',
                                'feat10','feat11','feat12','feat13','feat14','feat14','feat15','feat16','feat17',
                                'feat18','feat19','feat20','feat21','feat22','feat23','feat24','feat25','feat26',
                                'feat27','feat28','feat29', 'CC1', 'CC2', 'CC3','CC4', 'CC5','Base_Time', 
                                'Post_Length','Post Share Count','Post Promotion Status', 'H local'])),
    ('missing_trt', MissingValues())
])

data_pipe = FeatureUnion([
    ('obj_to_dummy', p1),
    ('obj_to_num', p2),
    ('obj_to_cyclic', p3),
    ('num_miss', p4)
])

In [9]:
len(data_pipe.get_feature_names())

0

In [10]:
X_train,X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [11]:
data_pipe.fit(X_train)


FeatureUnion(transformer_list=[('obj_to_dummy',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['page_category'])),
                                                  ('missing_trt',
                                                   DataFrameImputer()),
                                                  ('create_dummy',
                                                   dummy(freq_cutoff=200))])),
                               ('obj_to_num',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Post '
                                                                              'Published '
                                                                              'Weekday',
                                                                              'Base '
                                          

In [12]:
len(data_pipe.get_feature_names())

75

In [13]:
data_pipe.get_feature_names()

['obj_to_dummy__page_category_Professional sports team',
 'obj_to_dummy__page_category_Musician/band',
 'obj_to_dummy__page_category_Artist',
 'obj_to_dummy__page_category_Political party',
 'obj_to_dummy__page_category_Community',
 'obj_to_dummy__page_category_Movie',
 'obj_to_dummy__page_category_Actor/director',
 'obj_to_dummy__page_category_University',
 'obj_to_dummy__page_category_Athlete',
 'obj_to_dummy__page_category_Public figure',
 'obj_to_dummy__page_category_Product/service',
 'obj_to_dummy__page_category_Non-profit organization',
 'obj_to_dummy__page_category_Church/religious organization',
 'obj_to_dummy__page_category_Record label',
 'obj_to_dummy__page_category_Company',
 'obj_to_dummy__page_category_News/media website',
 'obj_to_dummy__page_category_Personal blog',
 'obj_to_dummy__page_category_Entertainer',
 'obj_to_dummy__page_category_Education',
 'obj_to_dummy__page_category_Author',
 'obj_to_dummy__page_category_Entertainment website',
 'obj_to_dummy__page_catego

In [14]:
x_train = pd.DataFrame(data=data_pipe.transform(X_train),
                    columns=data_pipe.get_feature_names())
x_test=pd.DataFrame(data=data_pipe.transform(X_test),
                    columns=data_pipe.get_feature_names())

In [15]:
x_train

,obj_to_dummy__page_category_Professional sports team,obj_to_dummy__page_category_Musician/band,obj_to_dummy__page_category_Artist,obj_to_dummy__page_category_Political party,obj_to_dummy__page_category_Community,obj_to_dummy__page_category_Movie,obj_to_dummy__page_category_Actor/director,obj_to_dummy__page_category_University,obj_to_dummy__page_category_Athlete,obj_to_dummy__page_category_Public figure,...,num_miss__CC1,num_miss__CC2,num_miss__CC3,num_miss__CC4,num_miss__CC5,num_miss__Base_Time,num_miss__Post_Length,num_miss__Post Share Count,num_miss__Post Promotion Status,num_miss__H local
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,14.0,13.0,7.0,0.0,24.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,85.0,19.0,66.0,83.0,-47.0,31.0,114.0,220.0,0.0,24.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,-1.0,57.0,0.0,21.0,0.0,24.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,132.0,1.0,0.0,24.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,6.0,0.0,59.0,64.0,1.0,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,3.0,2.0,4.0,1.0,41.0,0.0,1.0,0.0,24.0
32755,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.0,1.0,1.0,2.0,0.0,37.0,54.0,1.0,0.0,24.0
32756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,-1.0,52.0,0.0,16.0,0.0,24.0
32757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,72.0,127.0,1.0,0.0,24.0


In [16]:
x_test

,obj_to_dummy__page_category_Professional sports team,obj_to_dummy__page_category_Musician/band,obj_to_dummy__page_category_Artist,obj_to_dummy__page_category_Political party,obj_to_dummy__page_category_Community,obj_to_dummy__page_category_Movie,obj_to_dummy__page_category_Actor/director,obj_to_dummy__page_category_University,obj_to_dummy__page_category_Athlete,obj_to_dummy__page_category_Public figure,...,num_miss__CC1,num_miss__CC2,num_miss__CC3,num_miss__CC4,num_miss__CC5,num_miss__Base_Time,num_miss__Post_Length,num_miss__Post Share Count,num_miss__Post Promotion Status,num_miss__H local
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,236.0,19.0,60.0,194.0,-41.0,61.0,11.0,1.0,0.0,24.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,3.0,3.0,-1.0,49.0,640.0,24.0,0.0,24.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,108.0,12.0,20.0,92.0,-8.0,62.0,149.0,41.0,0.0,24.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,384.0,257.0,127.0,375.0,130.0,26.0,412.0,466.0,0.0,24.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,2.0,3.0,14.0,-1.0,54.0,315.0,19.0,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,42.0,137.0,1.0,0.0,24.0
8186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,3.0,4.0,-3.0,54.0,176.0,1.0,0.0,24.0
8187,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,1.0,4.0,3.0,-3.0,66.0,705.0,36.0,0.0,24.0
8188,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,-1.0,40.0,0.0,25.0,0.0,24.0


### Linear Regression

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [18]:
lm=LinearRegression()

In [19]:
cv_mae=-cross_val_score(lm,
                        x_train,y_train,
                        cv=10,
                        scoring='neg_mean_absolute_error')

In [20]:
cv_mae

array([8.41218356, 7.64909539, 8.8121442 , 8.20896014, 8.19405407,
       8.20330916, 8.48150943, 9.30766375, 9.85185442, 9.05894654])

In [21]:
cv_mae.mean()

8.617972065100641

In [22]:
cv_mae.std()

0.6122856445605301

In [23]:
lm.fit(x_train,y_train)

LinearRegression()

In [24]:
lm.intercept_

array([-2.84854125])

In [25]:
lm.coef_

array([[ 3.67304355e-01, -9.05784241e-01, -9.54177840e-02,
        -1.26097165e+00, -4.18285347e-01,  2.70225099e+00,
        -5.94996513e-02, -1.51301136e-01,  9.16816726e-01,
         2.75764602e-02,  4.01476110e-01, -5.42719345e-01,
        -6.36709145e-01, -5.61021486e-02,  1.58124886e-01,
        -3.67166380e-01, -4.83414636e-01,  1.68194088e-01,
        -8.81548687e-02,  1.95898552e-01, -6.01723735e-02,
         3.80535188e+00, -4.22915788e-01,  5.66219635e+00,
        -2.71774726e-01, -4.48586266e+00, -1.20682726e+00,
        -1.14674036e-02, -4.17425065e-01, -1.78209026e-01,
        -1.02484039e-01,  5.14939938e-02, -5.57619255e-01,
        -6.83888423e-01,  5.07423104e-01, -2.99355797e-01,
         5.51974098e-08, -2.74466968e-06, -2.05061006e-05,
        -4.99954804e-01, -1.84362355e-03,  7.30776117e-01,
         3.25266146e-01,  1.09157966e-01, -9.04305360e-02,
        -4.78837848e-03,  1.47425112e-01,  1.42018787e-01,
        -1.77273860e-02, -1.77273860e-02,  1.36422431e-0

In [26]:
list(zip(x_train.columns,lm.coef_))

[('obj_to_dummy__page_category_Professional sports team',
  array([ 3.67304355e-01, -9.05784241e-01, -9.54177840e-02, -1.26097165e+00,
         -4.18285347e-01,  2.70225099e+00, -5.94996513e-02, -1.51301136e-01,
          9.16816726e-01,  2.75764602e-02,  4.01476110e-01, -5.42719345e-01,
         -6.36709145e-01, -5.61021486e-02,  1.58124886e-01, -3.67166380e-01,
         -4.83414636e-01,  1.68194088e-01, -8.81548687e-02,  1.95898552e-01,
         -6.01723735e-02,  3.80535188e+00, -4.22915788e-01,  5.66219635e+00,
         -2.71774726e-01, -4.48586266e+00, -1.20682726e+00, -1.14674036e-02,
         -4.17425065e-01, -1.78209026e-01, -1.02484039e-01,  5.14939938e-02,
         -5.57619255e-01, -6.83888423e-01,  5.07423104e-01, -2.99355797e-01,
          5.51974098e-08, -2.74466968e-06, -2.05061006e-05, -4.99954804e-01,
         -1.84362355e-03,  7.30776117e-01,  3.25266146e-01,  1.09157966e-01,
         -9.04305360e-02, -4.78837848e-03,  1.47425112e-01,  1.42018787e-01,
         -1.772738

In [27]:
test_pred=lm.predict(x_test)

In [28]:
test_pred

array([[11.94807768],
       [-1.1456829 ],
       [-2.93523578],
       ...,
       [-3.08536899],
       [-0.6372374 ],
       [-4.64550826]])

### Ridge Regression

In [29]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV


In [30]:
lambdas=np.linspace(70,85,100)

In [31]:
lambdas

array([70.        , 70.15151515, 70.3030303 , 70.45454545, 70.60606061,
       70.75757576, 70.90909091, 71.06060606, 71.21212121, 71.36363636,
       71.51515152, 71.66666667, 71.81818182, 71.96969697, 72.12121212,
       72.27272727, 72.42424242, 72.57575758, 72.72727273, 72.87878788,
       73.03030303, 73.18181818, 73.33333333, 73.48484848, 73.63636364,
       73.78787879, 73.93939394, 74.09090909, 74.24242424, 74.39393939,
       74.54545455, 74.6969697 , 74.84848485, 75.        , 75.15151515,
       75.3030303 , 75.45454545, 75.60606061, 75.75757576, 75.90909091,
       76.06060606, 76.21212121, 76.36363636, 76.51515152, 76.66666667,
       76.81818182, 76.96969697, 77.12121212, 77.27272727, 77.42424242,
       77.57575758, 77.72727273, 77.87878788, 78.03030303, 78.18181818,
       78.33333333, 78.48484848, 78.63636364, 78.78787879, 78.93939394,
       79.09090909, 79.24242424, 79.39393939, 79.54545455, 79.6969697 ,
       79.84848485, 80.        , 80.15151515, 80.3030303 , 80.45

In [32]:
params={'alpha':lambdas}

In [33]:
model=Ridge()

In [34]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error',
                        verbose=20,n_jobs=-1)

In [35]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([70.        , 70.15151515, 70.3030303 , 70.45454545, 70.60606061,
       70.75757576, 70.90909091, 71.06060606, 71.21212121, 71.36363636,
       71.51515152, 71.66666667, 71.81818182, 71.96969697, 72.12121212,
       72.27272727, 72.42424242, 72.57575758, 72.72727273, 72.87878788,
       73.03030303, 73.18181818, 73.33333333, 73.484848...
       80.60606061, 80.75757576, 80.90909091, 81.06060606, 81.21212121,
       81.36363636, 81.51515152, 81.66666667, 81.81818182, 81.96969697,
       82.12121212, 82.27272727, 82.42424242, 82.57575758, 82.72727273,
       82.87878788, 83.03030303, 83.18181818, 83.33333333, 83.48484848,
       83.63636364, 83.78787879, 83.93939394, 84.09090909, 84.24242424,
       84.39393939, 84.54545455, 84.6969697 , 84.84848485, 85.        ])},
             scoring='neg_mean_absolute_error', verbose=20)

In [36]:
grid_search.best_estimator_

Ridge(alpha=85.0)

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([0.1429095 , 0.07215762, 0.07751305, 0.0904511 , 0.08617163,
        0.0775229 , 0.08030672, 0.11546917, 0.09074707, 0.08646357,
        0.08883684, 0.08424633, 0.08935938, 0.08461375, 0.08223221,
        0.10197177, 0.07585595, 0.08727286, 0.08626878, 0.09596083,
        0.09027119, 0.08024344, 0.09272971, 0.08738832, 0.07480891,
        0.07346244, 0.07736127, 0.07891359, 0.08621025, 0.07375212,
        0.07925236, 0.07345021, 0.07950902, 0.07268846, 0.0681706 ,
        0.07803633, 0.07731943, 0.07235873, 0.07392545, 0.07017143,
        0.09721019, 0.08267186, 0.07981637, 0.11560755, 0.08582571,
        0.08111057, 0.10648623, 0.09939179, 0.07068896, 0.11224256,
        0.09541867, 0.10500743, 0.08584039, 0.08018646, 0.12592497,
        0.09354815, 0.11687624, 0.07604387, 0.10313597, 0.07591968,
        0.09992266, 0.06737983, 0.08677497, 0.09287519, 0.09418766,
        0.08265953, 0.09799454, 0.11700649, 0.08542755, 0.07304349,
        0.0661411 , 0.08525651,

In [38]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [39]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: -8.607608 (std: 0.613703)
Parameters: {'alpha': 85.0}

Model with rank: 2
Mean validation score: -8.607620 (std: 0.613701)
Parameters: {'alpha': 84.84848484848484}

Model with rank: 3
Mean validation score: -8.607632 (std: 0.613699)
Parameters: {'alpha': 84.6969696969697}

Model with rank: 4
Mean validation score: -8.607645 (std: 0.613697)
Parameters: {'alpha': 84.54545454545455}

Model with rank: 5
Mean validation score: -8.607657 (std: 0.613695)
Parameters: {'alpha': 84.39393939393939}



In [40]:
test_pred=grid_search.predict(x_test)

### Coefficients

In [41]:
grid_search.best_estimator_

Ridge(alpha=85.0)

In [42]:
ridge_model=grid_search.best_estimator_

In [43]:
ridge_model.fit(x_train,y_train)

Ridge(alpha=85.0)

In [44]:
list(zip(data_pipe.get_feature_names(),ridge_model.coef_))

[('obj_to_dummy__page_category_Professional sports team',
  array([ 2.25682046e-01, -9.30859268e-01, -1.40491557e-01, -1.24569575e+00,
         -4.38444319e-01,  2.51116514e+00, -9.39362986e-02, -1.52616612e-01,
          7.95198965e-01,  1.40118495e-02,  3.41922896e-01, -4.76904760e-01,
         -5.61277391e-01, -6.46467748e-02,  1.27483385e-01, -3.26677549e-01,
         -4.03686459e-01,  1.08563973e-01, -7.55065708e-02,  1.49897537e-01,
         -5.34167239e-02,  2.85123586e+00, -3.44177974e-01,  4.12138078e+00,
         -2.16420646e-01, -3.27444601e+00, -8.88135046e-01, -9.00612202e-02,
         -3.05177754e-01, -1.63473775e-01, -9.94984405e-02,  4.97248189e-02,
         -5.45504853e-01, -6.76407109e-01,  4.98767169e-01, -2.96250548e-01,
          5.04560633e-08, -3.01127321e-06, -1.99625995e-05, -4.93802821e-01,
         -1.64295943e-03,  7.25776067e-01,  3.19246681e-01,  1.08123885e-01,
         -8.86336006e-02, -4.97133647e-03,  1.45117846e-01,  1.42983111e-01,
         -1.658502

In [45]:
lm.coef_/ridge_model.coef_

array([[ 1.62753024,  0.97306249,  0.67917095,  1.01226295,  0.95402159,
         1.0760945 ,  0.63340426,  0.99138052,  1.15294004,  1.96808138,
         1.17417147,  1.13800362,  1.134393  ,  0.86782595,  1.24035682,
         1.12394127,  1.19750025,  1.54926246,  1.16751255,  1.30688306,
         1.12647069,  1.33463244,  1.22877063,  1.37385907,  1.25577079,
         1.36996079,  1.35883305,  0.12732898,  1.36780961,  1.09013832,
         1.03000649,  1.03557931,  1.02220769,  1.01106037,  1.01735466,
         1.01048184,  1.0939698 ,  0.91146485,  1.02722597,  1.01245838,
         1.12213577,  1.00688925,  1.01885521,  1.00956386,  1.02027375,
         0.96319742,  1.01589926,  0.99325568,  1.06887877,  1.06887878,
         1.01335657,  0.98694472,  1.01237892, -1.85334086,  1.00556029,
         1.01184878,  1.21281153,  1.0088962 ,  1.01156064,  1.00777458,
         0.95693423,  0.98028295,  1.06448948,  0.97885266,  1.00584742,
         0.99600997,  1.000183  ,  1.00059703,  0.9

### Lasso Regression

In [46]:
lambdas=np.linspace(80,85,100)

model=Lasso(fit_intercept=True)

params={'alpha':lambdas}

In [47]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error',
                        verbose=20,n_jobs=-1)

In [48]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([80.        , 80.05050505, 80.1010101 , 80.15151515, 80.2020202 ,
       80.25252525, 80.3030303 , 80.35353535, 80.4040404 , 80.45454545,
       80.50505051, 80.55555556, 80.60606061, 80.65656566, 80.70707071,
       80.75757576, 80.80808081, 80.85858586, 80.90909091, 80.95959596,
       81.01010101, 81.06060606, 81.11111111, 81.16161616,...
       83.53535354, 83.58585859, 83.63636364, 83.68686869, 83.73737374,
       83.78787879, 83.83838384, 83.88888889, 83.93939394, 83.98989899,
       84.04040404, 84.09090909, 84.14141414, 84.19191919, 84.24242424,
       84.29292929, 84.34343434, 84.39393939, 84.44444444, 84.49494949,
       84.54545455, 84.5959596 , 84.64646465, 84.6969697 , 84.74747475,
       84.7979798 , 84.84848485, 84.8989899 , 84.94949495, 85.        ])},
             scoring='neg_mean_absolute_error', verbose=20)

In [49]:
grid_search.best_estimator_

Lasso(alpha=81.71717171717172)

In [50]:
lasso_model=grid_search.best_estimator_

In [51]:
lasso_model.fit(x_train,y_train)

Lasso(alpha=81.71717171717172)

In [52]:
lasso_model.intercept_

array([0.87597206])

In [53]:
list(zip(data_pipe.get_feature_names(),lasso_model.coef_))

[('obj_to_dummy__page_category_Professional sports team', -0.0),
 ('obj_to_dummy__page_category_Musician/band', -0.0),
 ('obj_to_dummy__page_category_Artist', 0.0),
 ('obj_to_dummy__page_category_Political party', 0.0),
 ('obj_to_dummy__page_category_Community', -0.0),
 ('obj_to_dummy__page_category_Movie', 0.0),
 ('obj_to_dummy__page_category_Actor/director', 0.0),
 ('obj_to_dummy__page_category_University', 0.0),
 ('obj_to_dummy__page_category_Athlete', 0.0),
 ('obj_to_dummy__page_category_Public figure', 0.0),
 ('obj_to_dummy__page_category_Product/service', 0.0),
 ('obj_to_dummy__page_category_Non-profit organization', 0.0),
 ('obj_to_dummy__page_category_Church/religious organization', -0.0),
 ('obj_to_dummy__page_category_Record label', -0.0),
 ('obj_to_dummy__page_category_Company', -0.0),
 ('obj_to_dummy__page_category_News/media website', -0.0),
 ('obj_to_dummy__page_category_Personal blog', -0.0),
 ('obj_to_dummy__page_category_Entertainer', 0.0),
 ('obj_to_dummy__page_catego

In [54]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: -6.965919 (std: 0.692314)
Parameters: {'alpha': 81.71717171717172}

Model with rank: 2
Mean validation score: -6.965920 (std: 0.692184)
Parameters: {'alpha': 81.66666666666667}

Model with rank: 3
Mean validation score: -6.965922 (std: 0.692445)
Parameters: {'alpha': 81.76767676767676}

Model with rank: 4
Mean validation score: -6.965923 (std: 0.692053)
Parameters: {'alpha': 81.61616161616162}

Model with rank: 5
Mean validation score: -6.965929 (std: 0.692577)
Parameters: {'alpha': 81.81818181818181}

